In [18]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [15]:
colunas = ["PAV", "AMBIENTE", "ÁREA", "TIPO", "ALTURA", "PÉ DIREITO ÚTIL", "REFLETÂNCIA", "FATOR DE DEPRECIAÇÃO", "NÍVEL DE ILUMINÂNCIA (LUX)", "FLUXO LUMINOSO TOTAL (LUMENS)", "COEFICIENTE DE UTILIZAÇÃO", "LÂMPADAS POR LUMINÁRIA", "FLUXO POR LÂMPADAS", "FLUXO POR LUMINÁRIA", "CÁLCULO DE LUMINÁRIAS", "TOTAL DE LUMINÁRIAS", "TOTAL DE LÂMPADAS", "POTÊNCIA UNITÁRIA", "MODELO DE LUMINÁRIA", "TEMPERATURA DA LÂMPADA", "POTENCIA TOTAL", "P. LIMITE A", "ETIQUETA FUTURA"]
df = pd.read_excel("PLANILHA_JF.xlsx", sheet_name=0, names=colunas)


In [ ]:
df = df[(df["ALTURA"].notnull()) & (df["ALTURA"] != "PLANTA CAD")].reset_index(drop=True).iloc[1:, :] # REMOVE LINHAS QUE NÃO TEM ALTURA e Remove a primeira linha que é parte do cabeçalho

In [ ]:
dfLista = df.values.tolist()  # Transforma o dataframe em lista
dfLista.pop(0)  # Remove o cabeçalho

# Filtra a lista para remover campos onde não há altura da sala
dfListaAntiga = list(filter(lambda x: True 
if (x[6] != "PLANTA CAD" 
and str(x[3]) != "nan" and str(x[3]) != "-" 
and str(x[4]) != "nan" and str(x[4]) != "-" and str(x[4]) != "10,87/9"
and str(x[5]) != "nan" and str(x[5]) != "-" 
and str(x[6]) != "nan" and str(x[6]) != "-" 
and str(x[0]) != "nan" and str(x[1]) != "nan") 
else False, dfLista))

dfListaNova = []

print(df.columns)
for x in dfListaAntiga:
    print(x)
    dfListaNova.append(x)


In [ ]:
dfNovo = pd.DataFrame(dfListaNova, columns=colunas)
dfNovo

In [ ]:
dfNovo = dfNovo[["PAV", "AMBIENTE", "COMPRIMENTO", "LARGURA", "ALTURA"]]

In [ ]:
import pandas as pd

# Lê a planilha "PLANILHA_JF.xlsx" e armazena em um DataFrame
df = pd.read_excel("PLANILHA_JF.xlsx")

# Define a lista colunas2 com os nomes das colunas
colunas2 = ["PAV", "AMBIENTE", "ÁREA", "TIPO", "ALTURA", "PÉ DIREITO ÚTIL", "REFLETÂNCIA", "FATOR DE DEPRECIAÇÃO", "NÍVEL DE ILUMINÂNCIA (LUX)", "FLUXO LUMINOSO TOTAL (LUMENS)", "COEFICIENTE DE UTILIZAÇÃO", "LÂMPADAS POR LUMINÁRIA", "FLUXO POR LÂMPADAS", "FLUXO POR LUMINÁRIA", "CÁLCULO DE LUMINÁRIAS", "TOTAL DE LUMINÁRIAS", "TOTAL DE LÂMPADAS", "POTÊNCIA UNITÁRIA", "MODELO DE LUMINÁRIA", "TEMPERATURA DA LÂMPADA", "POTENCIA TOTAL", "P. LIMITE A", "ETIQUETA FUTURA"]

# Define uma função para verificar e modificar os nomes das colunas
def verificar_colunas(colunas2, df):
    # Obtém o número de colunas na planilha
    n_colunas = len(df.columns)

    # Verifica se o número de nomes na lista colunas2 é igual ao número de colunas na planilha
    if len(colunas2) != n_colunas:
        # Percorre os nomes na lista colunas2
        for i, nome in enumerate(colunas2):
            # Verifica se o nome é diferente do nome da coluna correspondente na planilha
            if nome != df.columns[i]:
                # Se for diferente, coloca o nome da coluna na lista colunas2
                colunas2[i] = df.columns[i]
    
        # Imprime a lista colunas2 atualizada
        print("Lista colunas2:", colunas2)
        print("Lista Planilha:", df.columns)
    elif print("Planilhas iguais"):
        return

# Chama a função verificar_colunas com a lista colunas2 e o DataFrame df
verificar_colunas(colunas2, df)


In [ ]:
colunas2 = ["PAV", "AMBIENTE", "ÁREA", "TIPO", "ALTURA", "PÉ DIREITO ÚTIL", "REFLETÂNCIA", "FATOR DE DEPRECIAÇÃO", "NÍVEL DE ILUMINÂNCIA (LUX)", "FLUXO LUMINOSO TOTAL (LUMENS)", "COEFICIENTE DE UTILIZAÇÃO", "LÂMPADAS POR LUMINÁRIA", "FLUXO POR LÂMPADAS", "FLUXO POR LUMINÁRIA", "CÁLCULO DE LUMINÁRIAS", "TOTAL DE LUMINÁRIAS", "TOTAL DE LÂMPADAS", "POTÊNCIA UNITÁRIA", "MODELO DE LUMINÁRIA", "TEMPERATURA DA LÂMPADA", "POTENCIA TOTAL", "P. LIMITE A", "ETIQUETA FUTURA"]
df2 = pd.read_excel("PLANILHA_JF.xlsx", sheet_name=0, names=colunas2)
df2 = df2[df2["POTÊNCIA UNITÁRIA"].notnull() & (df2["POTÊNCIA UNITÁRIA"]!="SI")].reset_index(drop=True).iloc[1:,:] #remove colunas onde não tem potencia e remove a primeira linha que é parte do cabeçalho
df2 = df2[["PAV","AMBIENTE","ÁREA","POTÊNCIA UNITÁRIA","TOTAL DE LÂMPADAS"]]


In [ ]:
dfGeral = dfNovo.merge(df2,how="inner", left_on=["PAV","AMBIENTE"], right_on=["PAV","AMBIENTE"])

In [ ]:
campos = {"PAV": "pavimento", "AMBIENTE": "ambiente", "COMPRIMENTO":"comprimento", "LARGURA":"largura", "ÁREA": "area", "ALTURA": "distanciaPlanoDeTrabalhoTeto", "POT. (W)": "potenciaLampada", "QUANT.": "qntdLampadas"}
dfGeral = dfGeral.rename(columns=campos)
dfGeral.drop_duplicates(inplace=True)
dfGeral

In [ ]:
import requests

# Carga no Banco de Dados
for i in dfGeral.values.tolist():
    json = {"pavimento":str(i[0]),"ambiente":str(i[1]),"area":i[2],"comprimento":i[3],"distanciaPlanoDeTrabalhoTeto":i[4],"potenciaLampada":i[5],"qntdLampadas":[i][6]}
    response = requests.post(f"https://us-central1-eficiencia-energetica---jf.cloudfunctions.net/api/ambientes",json=json)
    print(f"PAV - {i[0]} Ambiente - {i[1]} ====> status:{response.status_code}")


### Geração do PDF de Qr-Code

In [ ]:
import requests
from fpdf import FPDF
import os

# save FPDF() class into a variable pdf
pdf = FPDF()

# set style and size of font that you want in the pdf
pdf.set_font("Arial", size=15)

# obter o diretório do trabalho atual
script_dir = os.getcwd()

for index, i in enumerate(dfGeral.values.tolist()):
    pavimento = i[0]
    ambiente = i[1]

    pdf.add_page()  # add a page

    # create a cell
    pdf.cell(w=0, h=100, txt=f"PAV. {pavimento} - {ambiente}",
             ln=1, align='C')

    response = requests.get(
        f"https://us-central1-eficiencia-energetica---jf.cloudfunctions.net/api/ambientesQrCode?pavimento={pavimento}&ambiente={ambiente}"
    ).json()

    qrCodeAmbiente = response["qrCodeAmbiente"]
    response2 = requests.get(qrCodeAmbiente)

    # nome do arquivo de imagem
    img_filename = f"qrCodeAmbiente{index}.png"

    # caminho completo do arquivo
    img_path = os.path.join(script_dir, img_filename)

    # salvar a imagem
    with open(img_path, "wb") as file:
        file.write(response2.content)

    # BytesIO(response2.content)
    pdf.image(img_path, x=pdf.w/2-50/2, y=pdf.h/2-50/2, w=50, h=50)

    # remover a imagem do diretório
    os.remove(img_path)

# nome do arquivo PDF
pdf_filename = "qrCodesAmbientes.pdf"

# caminho completo do arquivo PDF
pdf_path = os.path.join(script_dir, pdf_filename)

# salvar o PDF no mesmo diretório do script
pdf.output(pdf_path)
